#### 기본 Library 선언 및 Tensorflow 버전 확인

In [115]:
import tensorflow as tf
import numpy as np
import pandas as pd

tf.executing_eagerly()
tf.random.set_seed(777)

In [116]:
csv_text = pd.read_csv("./data/zoo.csv", index_col=0, header=None)
csv_text

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,,,,,,,,,,,,,,,,,
aardvark,1,0,0,1,0,0,1,1,1,1,0,0,4,0,0,1,1
antelope,1,0,0,1,0,0,0,1,1,1,0,0,4,1,0,1,1
bass,0,0,1,0,0,1,1,1,1,0,0,1,0,1,0,0,4
bear,1,0,0,1,0,0,1,1,1,1,0,0,4,0,0,1,1
boar,1,0,0,1,0,0,1,1,1,1,0,0,4,1,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
wallaby,1,0,0,1,0,0,0,1,1,1,0,0,2,1,0,1,1
wasp,1,0,1,0,1,0,0,0,0,1,1,0,6,0,0,0,6
wolf,1,0,0,1,0,0,1,1,1,1,0,0,4,1,0,1,1


In [117]:
xy = csv_text.to_numpy(dtype=np.int32)
x_data = xy[:, 0:-1]
y_data = xy[:, [-1]]

nb_classes = 7  # 0 ~ 6

Y_one_hot = tf.one_hot(list(y_data), nb_classes)
Y_one_hot = tf.reshape(Y_one_hot, [-1, nb_classes])

#### Weight & Bias setting

In [118]:
W = tf.Variable(tf.random.normal([16, nb_classes]), name="weight")
b = tf.Variable(tf.random.normal([nb_classes]), name="bias")
variables = [W, b]

#### tf.nn.softmax는 softmax activations를 계산해주는 함수
- softmax = exp(logits) / reduce_sum(exp(logits), dim)

In [119]:
def logit_function(X):
    return tf.matmul(X, W) + b

def hypothesis(X):
    return tf.nn.softmax(logit_function(X))

def cost_function(X, Y):
    logits = logit_function(X)
    cost_i = tf.compat.v1.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=Y)
    cost = tf.reduce_mean(cost_i)
    return cost

def gradient_function(X, Y):
    with tf.GradientTape() as tape:
        loss = cost_function(X, Y)
        grads = tape.gradient(loss, variables)
        return grads

def prediction(X, Y):
    pred = tf.argmax(hypothesis(X), 1)
    correct_prediction = tf.equal(pred, tf.argmax(Y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    return accuracy

def fit(X, Y, epochs=1000, verbose=100):
    optimizer = tf.compat.v1.train.GradientDescentOptimizer(learning_rate=0.1)
    for i in range(epochs):
        grads = gradient_function(X, Y)
        optimizer.apply_gradients(zip(grads, variables))
        if (i==0) | ((i+1) % verbose == 0):
            accuracy = prediction(X, Y).numpy()
            loss = cost_function(X, Y).numpy()
            print("Steps : {}, Loss : {}, Acc : {}".format(i+1, loss, accuracy))

x_data = tf.convert_to_tensor(x_data, dtype=tf.float32)
fit(x_data, Y_one_hot)

Steps : 1, Loss : 4.377018928527832, Acc : 0.039603959769010544
Steps : 100, Loss : 0.4108406603336334, Acc : 0.7821782231330872
Steps : 200, Loss : 0.2507157623767853, Acc : 0.8514851331710815
Steps : 300, Loss : 0.19064858555793762, Acc : 0.8811880946159363
Steps : 400, Loss : 0.15612542629241943, Acc : 0.8811880946159363
Steps : 500, Loss : 0.13297539949417114, Acc : 0.8910890817642212
Steps : 600, Loss : 0.11616460233926773, Acc : 0.8910890817642212
Steps : 700, Loss : 0.10335730761289597, Acc : 0.8910890817642212
Steps : 800, Loss : 0.09328297525644302, Acc : 0.8910890817642212
Steps : 900, Loss : 0.08517007529735565, Acc : 0.8910890817642212
Steps : 1000, Loss : 0.07851335406303406, Acc : 0.8910890817642212
